In [10]:
import pandas as pd
import json
import os

folder_path = "Sample Data"
file_name = "exchange_rates.csv"
file_path = os.path.join(folder_path, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    df = pd.read_csv(f)
print(df)
# Convert column names dynamically
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()            # Convert to lowercase
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
)

              Date Source Currency Target Currency  Exchange Rate
0       2024-01-01             EUR             USD         1.4930
1       2024-01-01             GBP             USD         0.8467
2       2024-01-01             INR             USD         1.1532
3       2024-01-01             JPY             USD         0.7640
4       2024-01-01             AUD             USD         0.5242
...            ...             ...             ...            ...
147925  2050-12-31             SGD             USD         1.0694
147926  2050-12-31             HKD             USD         1.2395
147927  2050-12-31             KRW             USD         1.4668
147928  2050-12-31             BRL             USD         1.1051
147929  2050-12-31             ZAR             USD         0.8718

[147930 rows x 4 columns]


In [12]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [ ]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS task_1_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS task_1_mg")
        cs.execute("USE WAREHOUSE task_1_warehouse_mg")
        cs.execute("USE SCHEMA task_1_mg")
        runQueries(cs,df)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [14]:
def runQueries(cs,df):    
    # Rename specific columns
    #df["customer_id"] = df["customer_id"].astype(str)
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_2"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))